In [1]:
import os
os.chdir("/home/ec2-user/scVI/")
os.getcwd()

from scvi.dataset.MouseBrain import ZeiselMoleArchData


In [33]:

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from umap import UMAP

use_cuda = True
import torch

In [3]:
from scvi.inference.annotation import compute_accuracy_tuple

# import scVI models


In [4]:

from scvi.inference import UnsupervisedTrainer, SemiSupervisedTrainer
from scvi.models.scanvi import SCANVI
from scvi.models.vae import VAE

In [5]:
import numpy as np

In [7]:
gene_dataset = ZeiselMoleArchData()

In [8]:
gene_dataset_fine = ZeiselMoleArchData(coarse=False)

In [9]:
print("The number of classes in the coarse data is %i\n" % len(np.unique(gene_dataset.labels)))
print("The number of classes in the fine data is %i\n" % len(np.unique(gene_dataset_fine.labels)))
if np.sum([gene_dataset_fine.labels_groups[x] for x in gene_dataset_fine.labels.ravel()] == gene_dataset.labels.ravel()) == len(gene_dataset_fine):
    print("the mapping of fine to coarse labels is Correct\n") 
else: print("check that labels_groups is right")
     


The number of classes in the coarse data is 39

The number of classes in the fine data is 265

the mapping of fine to coarse labels is Correct



In [10]:
import pandas as pd
genes1 = pd.read_table('../Zeisel/Zeisel.1.hvg_info.csv', delimiter=',')
geneid1 = np.asarray([x.replace('gene_', '') for x in genes1[genes1.keys()[0]]]).astype('int')
genenames1 = genes1['genename']
assert np.sum(np.asarray(genenames1) == gene_dataset.gene_names) == len(gene_dataset.gene_names)
genes = gene_dataset.gene_names[geneid1[:1000]]
from scvi.harmonization.utils_chenling import subsetByGenenames
gene_dataset = subsetByGenenames(gene_dataset,genes)
gene_dataset_fine = subsetByGenenames(gene_dataset_fine,genes)

Downsampling from 27998 to 1001 genes
Downsampling from 160796 to 160796 cells
Downsampling from 27998 to 1001 genes
Downsampling from 160796 to 160796 cells


In [11]:
vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches, n_labels=gene_dataset.n_labels,
          n_hidden=128, n_latent=10, n_layers=2, dispersion='gene')
trainer = UnsupervisedTrainer(vae, gene_dataset, train_size=1.0)
# trainer.model.load_state_dict(torch.load('../Zeisel/vae.pkl'))
# trainer.model.eval()
# trainer.train(n_epochs=100)
# torch.save(trainer.model.state_dict(),'../Zeisel/vae.pkl')

VAE(
  (z_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (linear_0): Linear(in_features=1001, out_features=128, bias=True)
          (batch_norm_0): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (non_linearity_0): ReLU()
        )
        (Layer 1): Sequential(
          (linear_1): Linear(in_features=128, out_features=128, bias=True)
          (batch_norm_1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (non_linearity_1): ReLU()
          (dropout_1): Dropout(p=0.1)
        )
      )
    )
    (mean_encoder): Linear(in_features=128, out_features=10, bias=True)
    (var_encoder): Linear(in_features=128, out_features=10, bias=True)
  )
  (l_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (linear_0): Linear(in_features=1001, out_features=128, bias=True)
        

In [12]:
full = trainer.create_posterior(trainer.model, gene_dataset, indices=np.arange(len(gene_dataset)))

# SCANVI coarse

In [13]:
scanvi = SCANVI(gene_dataset.nb_genes, gene_dataset.n_batches, gene_dataset.n_labels, n_layers=2)
scanvi.load_state_dict(full.model.state_dict(), strict=False)
trainer_scanvi = SemiSupervisedTrainer(scanvi, gene_dataset, classification_ratio=50,
                                       n_epochs_classifier=1, lr_classification=5 * 1e-3)
# trainer_scanvi.train(n_epochs=5)
# torch.save(trainer_scanvi.model.state_dict(),'../Zeisel/scanvi.coarse.pkl')
trainer_scanvi.model.load_state_dict(torch.load('../Zeisel/scanvi.coarse.pkl'))
trainer_scanvi.model.eval()
full_coarse = trainer_scanvi.create_posterior(trainer_scanvi.model, gene_dataset, indices=np.arange(len(gene_dataset)))


# SCANVI Fine

In [38]:
K=5

In [87]:
# ,'dropout_rate':0.2
scanvi = SCANVI(gene_dataset_fine.nb_genes, gene_dataset_fine.n_batches, gene_dataset_fine.n_labels, n_layers=2,
                classifier_parameters = {'dropout_rate':0.2, 'n_hidden':256, 'n_layers':2})
scanvi.load_state_dict(full.model.state_dict(), strict=False)
trainer_scanvi = SemiSupervisedTrainer(scanvi, gene_dataset_fine, classification_ratio=100,
                                       n_epochs_classifier=1, lr_classification=5 * 1e-3,n_labelled_samples_per_class=K)
trainer_scanvi.train(n_epochs=5)
torch.save(trainer_scanvi.model.state_dict(),'../Zeisel/scanvi.%i.pkl'%K)
trainer_scanvi.model.load_state_dict(torch.load('../Zeisel/scanvi.%i.pkl'%K))
trainer_scanvi.model.eval()
full_fine = trainer_scanvi.create_posterior(trainer_scanvi.model, gene_dataset_fine, indices=np.arange(len(gene_dataset_fine)))

training: 100%|██████████| 5/5 [22:03<00:00, 264.77s/it]


In [88]:
unlabelled_idx= trainer_scanvi.unlabelled_set.indices
labelled_idx= trainer_scanvi.labelled_set.indices

In [89]:
_,SCANVI_fine_pred = full_fine.sequential().compute_predictions()

In [90]:
len(np.unique(gene_dataset_fine.labels[unlabelled_idx]))

265

In [91]:
len(np.unique(SCANVI_fine_pred[unlabelled_idx]))

166

In [92]:
scanvi_fine_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],SCANVI_fine_pred[unlabelled_idx])
scanvi_fine_acc[1]

0.32015172465944486

# SCANVI Hierarchical

In [94]:
scanvi = SCANVI(gene_dataset_fine.nb_genes, gene_dataset_fine.n_batches, gene_dataset_fine.n_labels, n_layers=2,
                classifier_parameters = {'dropout_rate':0.2, 'n_hidden':256, 'n_layers':2},
               labels_groups=gene_dataset_fine.labels_groups, use_labels_groups=True)

scanvi.load_state_dict(full.model.state_dict(), strict=False)


trainer_scanvi = SemiSupervisedTrainer(scanvi, gene_dataset_fine, classification_ratio=100,
                                       n_epochs_classifier=1, lr_classification=5 * 1e-3,n_labelled_samples_per_class=K)

trainer_scanvi.train(n_epochs=5)
torch.save(trainer_scanvi.model.state_dict(),'../Zeisel/scanvi.hier.%i.pkl'%K)
trainer_scanvi.model.load_state_dict(torch.load('../Zeisel/scanvi.hier.%i.pkl'%K))
trainer_scanvi.model.eval()
full_hier = trainer_scanvi.create_posterior(trainer_scanvi.model, gene_dataset_fine, indices=np.arange(len(gene_dataset_fine)))

training: 100%|██████████| 5/5 [35:37<00:00, 427.44s/it]


In [95]:
_,SCANVI_hier_pred = full_hier.sequential().compute_predictions()
scanvi_hier_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],SCANVI_hier_pred[unlabelled_idx])


In [96]:
len(np.unique(SCANVI_hier_pred[unlabelled_idx]))

227

In [97]:
scanvi_hier_acc[1]

0.3674891651661416

# scVI and Seurat latent space

In [51]:
vae_posterior = trainer.create_posterior(trainer.model, gene_dataset, indices=np.arange(len(gene_dataset)))
seurat_latent = np.genfromtxt('../Zeisel/Zeisel.1.CCA.txt')
latent, batch_indices, labels = vae_posterior.sequential().get_latent()
batch_indices = batch_indices.ravel()

# Coarse to Coarse

In [53]:
import copy
gene_dataset_coarse = copy.deepcopy(gene_dataset_fine)
gene_dataset_coarse.labels = gene_dataset_fine.labels_groups[gene_dataset_fine.labels.ravel()].reshape(len(gene_dataset_fine),1)
gene_dataset_coarse.cell_types = gene_dataset_fine.groups
gene_dataset_coarse.n_labels = gene_dataset_fine.groups.shape[0]

In [54]:
scanvi = SCANVI(gene_dataset.nb_genes, gene_dataset.n_batches, gene_dataset.n_labels, n_layers=2)
scanvi.load_state_dict(full.model.state_dict(), strict=False)
trainer_scanvi = SemiSupervisedTrainer(scanvi, gene_dataset, classification_ratio=50,
                                       n_epochs_classifier=1, lr_classification=5 * 1e-3)
labelled_idx = trainer_scanvi.labelled_set.indices
unlabelled_idx= trainer_scanvi.unlabelled_set.indices

In [55]:
from sklearn.neighbors import KNeighborsClassifier
def PartialPrediction(latent,labelled_idx,unlabelled_idx,labels):
        latent_labelled = latent[labelled_idx, :]
        latent_unlabelled = latent[unlabelled_idx, :]
        labels_labelled = labels[labelled_idx]
        labels_unlabelled = labels[unlabelled_idx]
        neigh = KNeighborsClassifier(n_neighbors=10)
        neigh = neigh.fit(latent_labelled, labels_labelled)
        labels_pred = neigh.predict(latent_unlabelled)
        return labels_pred


In [56]:
coarse_latent, _, _ = full_coarse.sequential().get_latent()

In [57]:
coarse_pred=(PartialPrediction(coarse_latent, labelled_idx,unlabelled_idx,gene_dataset.labels.ravel()))


In [58]:
# seurat coarse
seurat_coarse_pred=(PartialPrediction(seurat_latent, labelled_idx,unlabelled_idx,gene_dataset.labels.ravel()))
# scVI coarse
scVI_coarse_pred=(PartialPrediction(latent, labelled_idx,unlabelled_idx,gene_dataset.labels.ravel()))
coarse_pred=(PartialPrediction(coarse_latent, labelled_idx,unlabelled_idx,gene_dataset.labels.ravel()))
y,SCANVI_coarse_pred = full_coarse.sequential().compute_predictions()

In [61]:
seurat_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx], seurat_coarse_pred)
scVI_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],scVI_coarse_pred)
coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],coarse_pred)
scanvi_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],SCANVI_coarse_pred[unlabelled_idx])

In [62]:
seurat_coarse_acc[1]

0.748911306978098

In [63]:
scVI_coarse_acc[1]

0.8106541213164749

In [64]:
scanvi_coarse_acc[1]

0.8507258855655311

In [65]:
coarse_acc[1]

0.8471129374449987

# Fine to Fine

In [104]:
scanvi = SCANVI(gene_dataset_fine.nb_genes, gene_dataset_fine.n_batches, gene_dataset_fine.n_labels, n_layers=2)
scanvi.load_state_dict(full.model.state_dict(), strict=False)
trainer_scanvi = SemiSupervisedTrainer(scanvi, gene_dataset_fine, classification_ratio=50,
                                       n_epochs_classifier=1, lr_classification=5 * 1e-3,n_labelled_samples_per_class=K)
labelled_idx = trainer_scanvi.labelled_set.indices
unlabelled_idx= trainer_scanvi.unlabelled_set.indices

In [106]:
scanvi_latent,_,_ = full_fine.sequential().get_latent()

In [105]:
scanvi_hier_latent,_,_ = full_hier.sequential().get_latent()

In [107]:
len(labelled_idx)

1325

In [108]:
seurat_fine_pred=(PartialPrediction(seurat_latent, labelled_idx,unlabelled_idx,gene_dataset_fine.labels.ravel()))
scVI_fine_pred=(PartialPrediction(latent, labelled_idx,unlabelled_idx,gene_dataset_fine.labels.ravel()))
fine_pred=(PartialPrediction(scanvi_latent, labelled_idx,unlabelled_idx,gene_dataset_fine.labels.ravel()))
hier_pred=(PartialPrediction(scanvi_hier_latent, labelled_idx,unlabelled_idx,gene_dataset_fine.labels.ravel()))
scVI_fine_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],scVI_fine_pred)
seurat_fine_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],seurat_fine_pred)

# _,SCANVI_fine_pred = full_fine.sequential().compute_predictions()
# _,SCANVI_hier_pred = full_hier.sequential().compute_predictions()

In [109]:
scanvi_fine_acc[1]

0.32015172465944486

In [110]:
scanvi_hier_acc[1]

0.3674891651661416

In [111]:
scVI_fine_acc[1]

0.3226786444629229

In [112]:
seurat_fine_acc[1]

0.23375816599735355

In [113]:
fine_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],fine_pred)
fine_acc[1]

0.35548608982183777

In [114]:
hier_acc = compute_accuracy_tuple(gene_dataset_fine.labels.ravel()[unlabelled_idx],hier_pred)
hier_acc[1]

0.3626498253215884

In [79]:
print("%.3f & %.3f &%.3f &%.3f & %.3f & %.3f \\\\"%(seurat_fine_acc[0], scVI_fine_acc[0],scanvi_fine_acc[0],fine_acc[0],scanvi_hier_acc[0],hier_acc[0]))

0.390 & 0.489 &0.434 &0.523 & 0.460 & 0.537 \\


In [80]:
print("%.3f & %.3f &%.3f &%.3f & %.3f & %.3f \\\\"%(seurat_fine_acc[1], scVI_fine_acc[1],scanvi_fine_acc[1],fine_acc[1],scanvi_hier_acc[1],hier_acc[1]))

0.183 & 0.244 &0.367 &0.268 & 0.365 & 0.276 \\


# Fine to Coarse

In [103]:
len(SCANVI_hier_pred)

160796

In [104]:
len(fine_pred)

147546

In [105]:
seurat_fine_pred_coarse = gene_dataset_fine.labels_groups[seurat_fine_pred]
scVI_fine_pred_coarse = gene_dataset_fine.labels_groups[scVI_fine_pred]
SCANVI_fine_pred_coarse = gene_dataset_fine.labels_groups[SCANVI_fine_pred]
SCANVI_hier_pred_coarse = gene_dataset_fine.labels_groups[SCANVI_hier_pred]
fine_pred_coarse = gene_dataset_fine.labels_groups[fine_pred]
hier_pred_coarse = gene_dataset_fine.labels_groups[hier_pred]

In [106]:
seurat_fine_coarse_acc = compute_accuracy_tuple(seurat_fine_pred_coarse,gene_dataset.labels.ravel()[unlabelled_idx])
scVI_fine_coarse_acc = compute_accuracy_tuple(scVI_fine_pred_coarse,gene_dataset.labels.ravel()[unlabelled_idx])
scanvi_fine_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],SCANVI_fine_pred_coarse[unlabelled_idx])
scanvi_hier_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],SCANVI_hier_pred_coarse[unlabelled_idx])
fine_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],fine_pred_coarse)
hier_coarse_acc = compute_accuracy_tuple(gene_dataset.labels.ravel()[unlabelled_idx],hier_pred_coarse)


In [107]:
seurat_fine_coarse_acc

Accuracy(unweighted=0.8764859772545511, weighted=0.6984176331436392, worst=0.0, accuracy_classes=[0.9832838038632986, 0.7056195222546829, 0.3511450381679389, 0.9828009828009828, 0.5667608618862593, 0.29901960784313725, 0.6049279249078109, 0.486565231466494, 0.9993055555555556, 0.993006993006993, 0.9748603351955307, 0.10498687664041995, 0.3616822429906542, 0.9992414185473165, 0.9110130111524164, 0.9873801110550228, 0.7567191844300278, 0.8953488372093024, 0.9998979279371236, 0.3326703132769766, 0.9789132197891321, 0.5384615384615384, 0.8633440514469454, 0.6461538461538462, 0.997926386728875, 0.9901153212520593, 0.0, 0.24602510460251045, 0.13333333333333333, 0.48, 0.3904494382022472, 0.16883116883116883, 0.9664570230607966, 0.8755186721991701, 0.9602730819245774, 0.8426051560379919, 0.9779086892488954, 0.9820603424843708, 0.9036755386565273])

In [108]:
scVI_fine_coarse_acc

Accuracy(unweighted=0.9158093069280089, weighted=0.7870135333924407, worst=0.0, accuracy_classes=[0.9807682147197125, 0.8843712812376041, 0.4969040247678019, 0.9950738916256158, 0.8702236719478099, 0.45972495088408644, 0.5994563479373202, 0.625261688764829, 0.9990032891458188, 0.9147286821705426, 0.9495652173913044, 0.4115898959881129, 0.5399610136452242, 0.9952714204652923, 0.9418632936025298, 0.9949109414758269, 0.8086847073631215, 0.838810641627543, 0.994126035583322, 0.6031443544545021, 0.9087316522393677, 0.6363636363636364, 0.9847972972972973, 0.9230769230769231, 0.995334370139969, 0.9917491749174917, 0.0, 0.40271877655055227, 0.14674735249621784, 0.9672131147540983, 0.56951871657754, 0.53125, 0.9870609981515711, 0.9237492465340567, 0.9667590027700831, 0.9701917600151889, 0.984375, 0.9695716395864107, 0.9308755760368663])

In [109]:
scanvi_fine_coarse_acc

Accuracy(unweighted=0.8275317528092934, weighted=0.724645262288915, worst=0.0, accuracy_classes=[0.9971326926140286, 0.8816770812336222, 0.8633879781420765, 0.9926470588235294, 0.940009447331129, 0.0, 0.26778711484593837, 0.5720844811753902, 0.9970232188926375, 0.9197183098591549, 0.957542908762421, 0.26126126126126126, 0.7979041916167665, 0.9986729857819905, 0.886240520043337, 0.9924280666330136, 0.720886075949367, 0.9402597402597402, 0.7979396164830681, 0.6529801324503312, 0.996021483986473, 0.8571428571428571, 0.9949494949494949, 0.9033816425120773, 0.997926386728875, 0.9983443708609272, 0, 0.7093889716840537, 0.0, 0.9836065573770492, 0.0, 0.0, 0.9963636363636363, 0.8240146654445463, 0.9104715317672487, 0.9436097199035429, 0.9837157660991858, 0.0])

In [110]:
scanvi_hier_coarse_acc

Accuracy(unweighted=0.8561804454204114, weighted=0.7094155594111983, worst=0.0, accuracy_classes=[0.9928317315350714, 0.893569844789357, 0.9398907103825137, 0.9926470588235294, 0.6195087387812943, 0.0, 0.0, 0.0, 0.9955348283389561, 0.995774647887324, 0.990063233965673, 0.6238738738738738, 0.9431137724550899, 0.9941232227488151, 0.9111592632719393, 0.97778899545684, 0.6776371308016877, 0.8714285714285714, 0.9925540595675234, 0.0, 0.925601750547046, 1.0, 1.0, 0.0, 0.9958527734577501, 0.9983443708609272, 0, 0.9001490312965723, 0.0, 0.9836065573770492, 0.0, 0.0, 0.9981818181818182, 0.8811051459997381, 0.9597861327569536, 0.9530699313670933, 0.9866765358993338, 0.9639175257731959])

In [111]:
fine_coarse_acc

Accuracy(unweighted=0.9133965000745532, weighted=0.8367118861104342, worst=0.0, accuracy_classes=[0.9901768172888016, 0.9244104011288047, 0.8387978142076503, 0.9926470588235294, 0.9034010392064242, 0.8267148014440433, 0.696171802054155, 0.5711662075298439, 0.9972216709664616, 0.9971830985915493, 0.991869918699187, 0.5945945945945946, 0.8308383233532934, 0.9982938388625593, 0.8446370530877573, 0.98889449772842, 0.8554852320675106, 0.7961038961038961, 0.9964640282877737, 0.5726269315673289, 0.9600159140640541, 1.0, 0.9848484848484849, 0.9468599033816425, 0.997926386728875, 0.9950331125827815, 0, 0.7570789865871833, 0.36727272727272725, 0.9836065573770492, 0.21813031161473087, 0.8, 0.96, 0.815110645541443, 0.9374893350776407, 0.9573363012428121, 0.9866765358993338, 0.9199674443841562])

In [112]:
hier_coarse_acc

Accuracy(unweighted=0.9091334227969583, weighted=0.833522873919627, worst=0.0, accuracy_classes=[0.9895396378696968, 0.9229993952832091, 0.860655737704918, 0.9877450980392157, 0.8717524799244214, 0.8194945848375451, 0.6625583566760037, 0.5439240893786348, 0.9976185751141099, 0.9943661971830986, 0.99096657633243, 0.6486486486486487, 0.8473053892215568, 0.9986729857819905, 0.8470205850487541, 0.9863705199394245, 0.8284810126582278, 0.8415584415584415, 0.996532027743778, 0.4794701986754967, 0.9669783170877263, 1.0, 0.9730639730639731, 0.9178743961352657, 0.997926386728875, 0.9950331125827815, 0, 0.7749627421758569, 0.4, 0.9672131147540983, 0.2563739376770538, 0.72, 0.9672727272727273, 0.8094801623674217, 0.9347591149536432, 0.9588202559821926, 0.9800148038490007, 0.938415626695605])